In [26]:
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain.schema import Document
from langchain_community.document_loaders import TextLoader
from pydantic import BaseModel,Field
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from dotenv import load_dotenv

load_dotenv() 
llm_model = ChatGoogleGenerativeAI(model = 'gemini-2.0-flash',max_retries=2)
embedding_model = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2') ##embedding model


In [9]:
##doc loader##
loader = TextLoader(
    file_path='data/langchain_sample.txt',
    encoding='utf-8'
)
doc = loader.load()

##text splitter##
chunker = RecursiveCharacterTextSplitter(
    chunk_size = 200,
    chunk_overlap = 50,
    separators = ["\n"," "]
)
doc_chunks = chunker.split_documents(doc)
print(f"Number of chunks created: {len(doc_chunks)}")
print(f"Chunks : ")
for index,chunk in enumerate(doc_chunks):
    print(f"Chunk number: {index+1}")
    print(f"Chunk content : {chunk.page_content}")

Number of chunks created: 45
Chunks : 
Chunk number: 1
Chunk content : LangChain is a flexible framework designed for developing applications powered by large language models (LLMs). It provides tools and abstractions to work with LLMs more effectively and includes
Chunk number: 2
Chunk content : to work with LLMs more effectively and includes components for prompt management, chains, memory, and agents.
Chunk number: 3
Chunk content : LangChain integrates with many third-party services such as OpenAI, Hugging Face, and Cohere. This enables developers to experiment with different models and optimize performance for specific use
Chunk number: 4
Chunk content : models and optimize performance for specific use cases like summarization, question answering, or translation.
Chunk number: 5
Chunk content : Retrieval-Augmented Generation (RAG) is a powerful technique where external knowledge is retrieved and passed into the prompt to ground LLM responses. LangChain makes it easy to implement R

In [13]:
## vectore store as retreiver ##
vector_store = FAISS.from_documents(
    documents=doc_chunks,
    embedding = embedding_model
)
retreiver = vector_store.as_retriever(search_kwargs={'k':5})

##retreiver test##
query = 'Agentic AI capabilities in Langchain'
retreiver.invoke(query)


[Document(id='bbda434d-2133-41aa-95a0-66d0e537da62', metadata={'source': 'data/langchain_sample.txt'}, page_content='Agents in LangChain are chains that use LLMs to decide which tools to use and in what order. This makes them suitable for multi-step tasks like question answering with search and code execution.'),
 Document(id='875f1dd8-a808-4411-ac26-c8d8db48abfd', metadata={'source': 'data/langchain_sample.txt'}, page_content='LangChain agents can operate in various modes such as zero-shot-react-description, plan-and-execute, or conversational-react-description.'),
 Document(id='50afd39c-8619-4f8c-8707-acc1d63b005d', metadata={'source': 'data/langchain_sample.txt'}, page_content='With strong community support, detailed documentation, and continuous innovation, LangChain has become a cornerstone of the modern AI application development stack.'),
 Document(id='71d10f58-997a-4883-a717-e067e644efeb', metadata={'source': 'data/langchain_sample.txt'}, page_content='LangChain is a flexible f

In [ ]:
instruction ='''
You are an intelligent re-ranking model. Your task is to carefully evaluate and reorder a \
list of retrieved documents based on their relevance to the user's query.
The documents will be numbered.
The response should be only the document numbers in re-ranked state.

Context_docuements : {context}
User_Query : {user_query}

'''
prompt = ChatPromptTemplate.from_template(
    instruction
)

class response_structure(BaseModel):
    re_ranked_response:str = Field(...,description="Respond with re-ranked document numbers, no text required")

In [19]:
from typing import List

def format_docs(document_list)->List[str]:
    '''Fomrat documents to create a context for the LLM, numbers will be added in the beginning of the chunks'''
    doc_list = []
    for index,doc in enumerate(document_list):
        indexed_doc = f"{index+1}. {doc}"
        doc_list.append(indexed_doc)

    return doc_list

In [27]:
rag_chain = (
    {'context':retreiver | format_docs, 'user_query':RunnablePassthrough()}
    | prompt
    | llm_model.with_structured_output(response_structure)
)

In [23]:
rag_chain

{
  context: VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x00000274811D4640>, search_kwargs={'k': 5})
           | RunnableLambda(format_docs)
}
| ChatPromptTemplate(input_variables=['context', 'user_query'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'user_query'], input_types={}, partial_variables={}, template="\nYou are an intelligent re-ranking model. Your task is to carefully evaluate and reorder a list of retrieved documents based on their relevance to the user's query.\nThe documents will be numbered.\nThe response should be only the numbers re-ranked.\n\nContext : {context}\nUser_Query : {user_query}\n\n"), additional_kwargs={})])
| RunnableBinding(bound=ChatGoogleGenerativeAI(model='models/gemini-2.0-flash', google_api_key=SecretStr('**********'), max_retries=2, client=<google.ai.generativelanguage_v1beta.servic

In [32]:
response = rag_chain.invoke('Agentic AI capabilities in Langchain')
response

response_structure(re_ranked_response='1, 2, 5, 4, 3')